# Overview of WRF Output Data

The first rule of data processing:

**"ALWAYS LOOK AT YOUR DATA"**

\- D. Shea

WRF can be configured in various ways and can have variables turned on and off.  If you run in to problems, it could be due to a variable missing.  If your plot doesn't look right, there could be a map projection issue.

Let's look at some WRF data...how do we do that?

There are numerous tools available to examine NetCDF data, from both outside and inside of Python.

- **ncdump** (used for this example)
- ncl_filedump
- netcdf4-python 
- xarray


## ncdump

ncdump is a program included with the NetCDF libraries that can be used to examine NetCDF data.

By supplying the '-h' option, only the data descriptions are returned.  Otherwise, you'll get all of the data values, which can span miles.

To run:

```
$ ncdump -h wrfout_d01_2005-08-28_00:00:00
```

<div id="nc_dims" style="font-size:75%;"/>
``` 
netcdf wrfout_d01_2005-08-28_00\:00\:00 {
dimensions:
    Time = UNLIMITED ; // (4 currently)
    
    DateStrLen = 19 ;
    
    west_east = 90 ;
    
    south_north = 73 ;
    
    bottom_top = 29 ;
    
    bottom_top_stag = 30 ;
    
    soil_layers_stag = 4 ;
    
    west_east_stag = 91 ;
    
    south_north_stag = 74 ;
    
```
</div>



<div id="nc_vars" style="font-size:75%;">
```
variables:
    char Times(Time, DateStrLen) ;
    float XLAT(Time, south_north, west_east) ;
        XLAT:FieldType = 104 ;
        XLAT:MemoryOrder = "XY " ;
        XLAT:description = "LATITUDE, SOUTH IS NEGATIVE" ;
        XLAT:units = "degree_north" ;
        XLAT:stagger = "" ;
        XLAT:coordinates = "XLONG XLAT" ;
    float XLONG(Time, south_north, west_east) ;
        XLONG:FieldType = 104 ;
        XLONG:MemoryOrder = "XY " ;
        XLONG:description = "LONGITUDE, WEST IS NEGATIVE" ;
        XLONG:units = "degree_east" ;
        XLONG:stagger = "" ;
        XLONG:coordinates = "XLONG XLAT" ;
        .
        .
        .
    float SST_INPUT(Time, south_north, west_east) ;
        SST_INPUT:FieldType = 104 ;
        SST_INPUT:MemoryOrder = "XY " ;
        SST_INPUT:description = "SEA SURFACE TEMPERATURE 
            FROM WRFLOWINPUT FILE" ;
        SST_INPUT:units = "K" ;
        SST_INPUT:stagger = "" ;
        SST_INPUT:coordinates = "XLONG XLAT XTIME" ;
```
</div>

<div id="nc_attrs" style="font-size:75%;">
```
// global attributes:
        :TITLE = " OUTPUT FROM WRF V3.7 MODEL" ;
        :START_DATE = "2005-08-28_00:00:00" ;
        :SIMULATION_START_DATE = "2005-08-28_00:00:00" ;
        :WEST-EAST_GRID_DIMENSION = 91 ;
        :SOUTH-NORTH_GRID_DIMENSION = 74 ;
        :BOTTOM-TOP_GRID_DIMENSION = 30 ;
        :DX = 30000.f ;
        :DY = 30000.f ;
        .
        .
        .
        :CEN_LAT = 28.00002f ;
        :CEN_LON = -89.f ;
        :TRUELAT1 = 30.f ;
        :TRUELAT2 = 60.f ;
        :MOAD_CEN_LAT = 28.00002f ;
        :STAND_LON = -89.f ;
        :POLE_LAT = 90.f ;
        :POLE_LON = 0.f ;
        :GMT = 0.f ;
        :JULYR = 2005 ;
        :JULDAY = 240 ;
        :MAP_PROJ = 1 ;
        :MAP_PROJ_CHAR = "Lambert Conformal" ;
        .
        .
        .
}
```
</div>

## Dimensions

WRF uses an Arakawa C-grid staggered grid [(taken from mmm website)] [1]

- Mass related quantities (pressure, temperature, etc) are computed at the center of a grid cell. 
- The u-component of the horizontal wind is calculated at the left and right edges of a grid cell.  It has one more 
  point in the x direction than the mass grid.
- The v-component of the horizontal wind is calculated at the bottom and top edges of a grid cell.  It has one more     point in the y direction than the mass grid. 
- The corners of each grid box are know as the 'staggered' grid, and has one additional point in both the x and y direction.

[1]: http://www2.mmm.ucar.edu/rt/amps/information/configuration/wrf_grid_structure.html

![alt](wrf_stagger.png)

<div id="nc_dims" style="font-size:75%;"/>
``` 
netcdf wrfout_d01_2005-08-28_00\:00\:00 {
dimensions:
    Time = UNLIMITED ; // (4 currently)
    
    DateStrLen = 19 ;
    
    west_east = 90 ;
    
    south_north = 73 ;
    
    bottom_top = 29 ;
    
    bottom_top_stag = 30 ; <-- Extra grid point
    
    soil_layers_stag = 4 ;
    
    west_east_stag = 91 ; <-- Extra grid point
    
    south_north_stag = 74 ; <-- Extra grid point
    
```
</div>

## Variables

- Each variable is made up of dimensions, attributes, and data values. 
- Pay special attention to the units and coordinates attribute.
  - The coordinates attribute specifies the variables that contain the latitude and longitude 
    information for each grid box (XLONG, XLAT).
  - If the domain is from a moving nest, then a time coordinate is also used (XTIME).
  - The coordinates are named in Fortran ordering, so they'll be listed in reverse.




<div id="var_example" style="font-size:75%;"/>
```
float P(Time, bottom_top, south_north, west_east) ; <- Dims
	P:FieldType = 104 ;                       <-  Attribute
	P:MemoryOrder = "XYZ" ;                   <-  Attribute
    P:description = "perturbation pressure" ; <-  Attribute
	P:units = "Pa" ;                          <-  Attribute
    P:stagger = "" ;                          <-  Attribute
	P:coordinates = "XLONG XLAT XTIME" ;      <-  Attribute
    
data:

 P =
  339.8281, 340.3281, 340.25, 341.4531, ... 
    355.8672, 356.9531, 361.2578, 365.7188,  ...

```
</div>

## Global Attributes

- Provide a description of how the model was set up (resolution, map projection, microphysics, etc)
- For plotting, the map projection parameters will be the most important.
- wrf-python uses this information to build the mapping object in your plotting system of choice - basemap, cartopy, pyngl.




<div id="proj_stuff" style="font-size:75%;"/>
```
.
.
.
:CEN_LAT = 28.00002f ;
:CEN_LON = -89.f ;
:TRUELAT1 = 30.f ;
:TRUELAT2 = 60.f ;
:MOAD_CEN_LAT = 28.00002f ;
:STAND_LON = -89.f ;
:POLE_LAT = 90.f ;
:POLE_LON = 0.f ;
.
.
:MAP_PROJ = 1 ;
:MAP_PROJ_CHAR = "Lambert Conformal" ;
.
.
.
```
</div>

## Your Turn!

In your workbook, modify the file_path variable to point to one of your WRF output NetCDF files and execute the cell to see the output of your file.  

In [1]:
from subprocess import Popen, PIPE, STDOUT

# Edit the line below to point to one of your WRF 
# files (full path)
file_path = "./wrfout_d01_2005-08-28_00:00:00"

# This simply executes 'ncdump -h {wrf_file}' 
# from Python
p = Popen(["ncdump", "-h", "{}".format(file_path)], 
          stdout=PIPE, stderr=STDOUT)
output, _ = p.communicate()

print (output)


netcdf wrfout_d01_2005-08-28_00\:00\:00 {
dimensions:
	Time = UNLIMITED ; // (4 currently)
	DateStrLen = 19 ;
	west_east = 90 ;
	south_north = 73 ;
	bottom_top = 29 ;
	bottom_top_stag = 30 ;
	soil_layers_stag = 4 ;
	west_east_stag = 91 ;
	south_north_stag = 74 ;
variables:
	char Times(Time, DateStrLen) ;
	float XLAT(Time, south_north, west_east) ;
		XLAT:FieldType = 104 ;
		XLAT:MemoryOrder = "XY " ;
		XLAT:description = "LATITUDE, SOUTH IS NEGATIVE" ;
		XLAT:units = "degree_north" ;
		XLAT:stagger = "" ;
		XLAT:coordinates = "XLONG XLAT" ;
	float XLONG(Time, south_north, west_east) ;
		XLONG:FieldType = 104 ;
		XLONG:MemoryOrder = "XY " ;
		XLONG:description = "LONGITUDE, WEST IS NEGATIVE" ;
		XLONG:units = "degree_east" ;
		XLONG:stagger = "" ;
		XLONG:coordinates = "XLONG XLAT" ;
	float LU_INDEX(Time, south_north, west_east) ;
		LU_INDEX:FieldType = 104 ;
		LU_INDEX:MemoryOrder = "XY " ;
		LU_INDEX:description = "LAND USE CATEGORY" ;
		LU_INDEX:units = "" ;
		LU_INDEX:stagger = "" ;

## Reading a WRF File in Python

You have several options to read a WRF NetCDF file in Python. 

- **netcdf4-python**
- PyNIO (currently Python 2.x only)
- xarray (Dataset not natively supported yet in wrf-python)


## netcdf4-python Example

``` python
from netCDF4 import Dataset

file_path = "./wrfout_d01_2005-08-28_00:00:00"

wrf_file = Dataset(file_path)

```

## Your Turn!

Modify the 'wrf_file' variable to point to your data.


In [2]:
from netCDF4 import Dataset

# Edit the line below to point to one of your WRF 
# files (full path)
file_path = "./wrfout_d01_2005-08-28_00:00:00"

wrf_file = Dataset(file_path)

print(wrf_file)


<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    TITLE:  OUTPUT FROM WRF V3.7 MODEL
    START_DATE: 2005-08-28_00:00:00
    SIMULATION_START_DATE: 2005-08-28_00:00:00
    WEST-EAST_GRID_DIMENSION: 91
    SOUTH-NORTH_GRID_DIMENSION: 74
    BOTTOM-TOP_GRID_DIMENSION: 30
    DX: 30000.0
    DY: 30000.0
    SKEBS_ON: 0
    SPEC_BDY_FINAL_MU: 0
    USE_Q_DIABATIC: 0
    GRIDTYPE: C
    DIFF_OPT: 1
    KM_OPT: 4
    DAMP_OPT: 0
    DAMPCOEF: 0.2
    KHDIF: 0.0
    KVDIF: 0.0
    MP_PHYSICS: 3
    RA_LW_PHYSICS: 1
    RA_SW_PHYSICS: 1
    SF_SFCLAY_PHYSICS: 1
    SF_SURFACE_PHYSICS: 2
    BL_PBL_PHYSICS: 1
    CU_PHYSICS: 1
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 3
    SST_UPDATE: 0
    GRID_FDDA: 0
    GFDDA_INTERVAL_M: 0
    GFDDA_END_H: 0
    GRID_SFDDA: 0
    SGFDDA_INTERVAL_M: 0
    SGFDDA_END_H: 0
    HYPSOMETRIC_OPT: 2
    USE_THETA_M: 0
    SF_URBAN_PHYSICS: 0
    SHCU_PHYSICS: 0
    MFSHCONV: 0
    FEEDBACK: 1
    SMOOTH_OPT

## Getting Variables and Attributes

netcdf4-python uses an old API that was originally created for an old package called Scientific.IO.NetCDF.  PyNIO also uses this API.  Some of it may look a little dated.




### Getting global attributes

The get the full dictionary of global attributes, use the \_\_dict\_\_ attribute.  To work with one attribute at a time, you can use the getncattr and setncattr methods. 

``` python
global_attrs = wrf_file.__dict__

# To get the value for MAP_PROJ, you can do:
map_proj = wrf_file.__dict__["MAP_PROJ"]

# Or more cleanly
map_prof = wrf_file.getncattr("MAP_PROJ")

```

### Getting variables, variable attributes, and variable data

All variables are stored in a dictionary attribute called *variables*.  

Let's get the perturbation pressure ("P") variable.

``` python

# This will return a netCDF4.Variable object
p = wrf_file.variables["P"]

```

To get the variable attributes, you can use the \_\_dict\_\_ attribute to get a dictionary of all attributes, or the *getncattr* function if you already know the attribute name.

``` python
# Return a dictionary of all of P's 
# attributes
p_attrs = p.__dict__

# Let's just get the 'coordinates' attribute
p_coords = p.getncattr("coordinates")

```

To get the variable's data as a numpy array, you need to use Python's [ ] API (\_\_getitem\_\_ for those that are more familiar with Python's data model).  

``` python

# Get a numpy array for all times
p_all_data = p[:,:,:,:]

# In numpy, there is implicit expansion of ':'
# across all dimensions. So, this is the 
# same as p[:,:,:,:]
p_all_data = p[:]

# You can also request specific values 
# by supplying indexes.  This will 
# extract the numpy array for time 
# index 0.

p_t0_data = p[0,:]


```

## Your Turn!

Modify 'file_path' to point to one of your WRF files.

In [3]:
from netCDF4 import Dataset

# Edit the line below to point to one of your WRF 
# files (full path)
file_path = "./wrfout_d01_2005-08-28_00:00:00"

# Create the netCDF4.Dataset object
wrf_file = Dataset(file_path)

# Get the global attribute dict
global_attrs = wrf_file.__dict__
print ("Global attributes for the file")
print(global_attrs)
print ("\n")

# Just get the 'MAP_PROJ' attribute
map_proj = wrf_file.getncattr("MAP_PROJ")
print ("The MAP_PROJ attribute:")
print (map_proj)
print("\n")

# Get the perturbation pressure variable
p = wrf_file.variables["P"]
print ("The P variable: ")
print(p)
print ("\n")

# Get the P attributes
p_attrs = p.__dict__
print ("The attribute dict for P")
print (p_attrs)
print ("\n")

# Get the 'coordinates' attribute for P
coords = p.getncattr("coordinates")
print ("Coordinates for P:")
print (coords)
print ("\n")

# Get the P numpy array for all times
p_all_data = p[:]
print ("The P numpy array: ")
print (p_all_data)
print ("\n")

# Get the P numpy array for time 0
p_t0_data = p[0,:]
print ("P array at time 0:")
print (p_t0_data)
print ("\n")

Global attributes for the file
OrderedDict([(u'TITLE', u' OUTPUT FROM WRF V3.7 MODEL'), (u'START_DATE', u'2005-08-28_00:00:00'), (u'SIMULATION_START_DATE', u'2005-08-28_00:00:00'), (u'WEST-EAST_GRID_DIMENSION', 91), (u'SOUTH-NORTH_GRID_DIMENSION', 74), (u'BOTTOM-TOP_GRID_DIMENSION', 30), (u'DX', 30000.0), (u'DY', 30000.0), (u'SKEBS_ON', 0), (u'SPEC_BDY_FINAL_MU', 0), (u'USE_Q_DIABATIC', 0), (u'GRIDTYPE', u'C'), (u'DIFF_OPT', 1), (u'KM_OPT', 4), (u'DAMP_OPT', 0), (u'DAMPCOEF', 0.2), (u'KHDIF', 0.0), (u'KVDIF', 0.0), (u'MP_PHYSICS', 3), (u'RA_LW_PHYSICS', 1), (u'RA_SW_PHYSICS', 1), (u'SF_SFCLAY_PHYSICS', 1), (u'SF_SURFACE_PHYSICS', 2), (u'BL_PBL_PHYSICS', 1), (u'CU_PHYSICS', 1), (u'SF_LAKE_PHYSICS', 0), (u'SURFACE_INPUT_SOURCE', 3), (u'SST_UPDATE', 0), (u'GRID_FDDA', 0), (u'GFDDA_INTERVAL_M', 0), (u'GFDDA_END_H', 0), (u'GRID_SFDDA', 0), (u'SGFDDA_INTERVAL_M', 0), (u'SGFDDA_END_H', 0), (u'HYPSOMETRIC_OPT', 2), (u'USE_THETA_M', 0), (u'SF_URBAN_PHYSICS', 0), (u'SHCU_PHYSICS', 0), (u'MFSHCON

## Pop Quiz

What is the first rule of data processing?

    A) YOU DO NOT TALK ABOUT DATA PROCESSING
    B) 60% OF THE TIME, IT WORKS EVERY TIME
    C) ALWAYS LOOK AT YOUR DATA

# wrf-python

wrf-python provides functionality similar to what is found in the NCL-WRF package:

- over 30 diagnostics calculations
- several interpolation routines (horizontal level, vertical cross section, horizontal "surface"
- plot helper utilities for cartopy, basemap, and PyNGL


The most commonly used functions:

- **getvar**: Extracts variables and diagnostic variables
- **interplevel**: Interpolates a variable to a horizontal plane at a specified level
- **vertcross**: Interpolates a 3D variable to a vertical cross section
- **vinterp**: Interpolates a variable to a new surface (e.g. theta-e)

## The *getvar* function

The *getvar* function can be used to:

- Extract NetCDF variables from a file, similar to netcdf4-python or PyNIO.
- Compute diagnostic variables
- Concatenate a variable (either NetCDF or diagnostic) across multiple files. 



### Simple *getvar* Example for HGT

``` python

from netCDF4 import Dataset
from wrf import getvar

file_path = "./wrfout_d01_2005-08-28_00:00:00"

wrf_file = Dataset(file_path)

hgt = getvar(wrf_file, "HGT", timeidx=0)

```

## Your Turn!

Modify 'file_path' to point to one of your WRF files.

In [4]:
from netCDF4 import Dataset
from wrf import getvar

file_path = "./wrfout_d01_2005-08-28_00:00:00"

wrf_file = Dataset(file_path)

hgt = getvar(wrf_file, "HGT", timeidx=0)

print(hgt)

<xarray.DataArray 'HGT' (south_north: 73, west_east: 90)>
array([[ 1545.590576,  1459.311279,  1364.495117, ...,   133.645905,
          167.242188,    79.194817],
       [ 1026.361206,   665.120422,   716.795898, ...,   309.192535,
          419.444214,   305.760345],
       [  482.241669,   355.454041,   624.357422, ...,   268.129059,
          375.759094,   339.033447],
       ..., 
       [ 1798.468018,  1730.704834,  1621.562012, ...,     0.      ,
            0.      ,     0.      ],
       [ 1856.451538,  1916.04126 ,  1809.998779, ...,     0.      ,
            0.      ,     0.      ],
       [ 1996.549927,  2086.711914,  1929.821045, ...,     0.      ,
            0.      ,     0.      ]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 -100.775 -100.514 -100.253 ...
    XLAT     (south_north, west_east) float32 17.8033 17.8397 17.8753 17.91 ...
    XTIME    float32 0.0
    Time     datetime64[ns] 2005-08-28
Dimensions without coordinates: south_north

## Computing a Diagnostic Variable with *getvar*

A list of available diagnostics is available at: http://wrf-python.readthedocs.io/en/latest/diagnostics.html

In this example, we're going to compute sea level pressure.

``` python

from netCDF4 import Dataset
from wrf import getvar

file_path = "./wrfout_d01_2005-08-28_00:00:00"

wrf_file = Dataset(file_path)

slp = getvar(wrf_file, "slp", timeidx=0)


```

## Your Turn!

Modify 'file_path' to point to one of your WRF files.

In [5]:
from netCDF4 import Dataset
from wrf import getvar

file_path = "./wrfout_d01_2005-08-28_00:00:00"

wrf_file = Dataset(file_path)

slp = getvar(wrf_file, "slp", timeidx=0)

print (slp)

<xarray.DataArray u'slp' (south_north: 73, west_east: 90)>
array([[ 1009.792908,  1009.460266,  1009.000244, ...,  1009.304504,
         1009.576172,  1009.437805],
       [ 1009.074402,  1008.862305,  1008.838928, ...,  1009.359436,
         1009.686279,  1009.900208],
       [ 1008.593506,  1009.354797,  1009.244751, ...,  1009.259644,
         1009.554932,  1009.780762],
       ..., 
       [ 1013.742432,  1012.939819,  1012.151184, ...,  1015.088684,
         1015.202271,  1015.308228],
       [ 1014.813171,  1014.403381,  1013.459473, ...,  1015.293518,
         1015.416077,  1015.525818],
       [ 1015.951599,  1015.64386 ,  1014.465393, ...,  1015.489502,
         1015.620789,  1015.736084]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 -100.775 -100.514 -100.253 ...
    XLAT     (south_north, west_east) float32 17.8033 17.8397 17.8753 17.91 ...
    XTIME    float32 0.0
    Time     datetime64[ns] 2005-08-28
Dimensions without coordinates: south_nort

## Combining Across Mulitple Files

wrf-python has two methods for combining a variable across multiple files

- **cat** - combines the the variable along the Time dimension (Note: you must put the files in the order you want them)
- **join** - creates a new left-most dimension for each file


To extract all times in to a single array, set *timeidx* to wrf.ALL_TIMES (an alias for None).  

In this example, we're using the 'cat' method, which is the most common.

``` python
from netCDF4 import Dataset
from wrf import getvar, ALL_TIMES

file_paths = [
     "./wrfout_d01_2005-08-28_00:00:00",
     "./wrfout_d01_2005-08-28_12:00:00"
     ]

wrf_files = [Dataset(file_paths[0]),
             Dataset(file_paths[1])]

slp = getvar(wrf_files, "slp", 
             timeidx=ALL_TIMES,
             method="cat")

```

## Your Turn!

Modify 'file_paths' to point to two of your WRF files.

### Example using the 'cat' method

In [6]:
from netCDF4 import Dataset
from wrf import getvar, ALL_TIMES

file_paths = ["./wrfout_d01_2005-08-28_00:00:00",
              "./wrfout_d01_2005-08-28_12:00:00"]

wrf_files = [Dataset(file_paths[0]),
             Dataset(file_paths[1])]

slp = getvar(wrf_files, "slp", timeidx=ALL_TIMES, method="cat")

print (slp)

<xarray.DataArray u'slp' (Time: 8, south_north: 73, west_east: 90)>
array([[[ 1009.792908,  1009.460266, ...,  1009.576172,  1009.437805],
        [ 1009.074402,  1008.862305, ...,  1009.686279,  1009.900208],
        ..., 
        [ 1014.813171,  1014.403381, ...,  1015.416077,  1015.525818],
        [ 1015.951599,  1015.64386 , ...,  1015.620789,  1015.736084]],

       [[ 1010.591248,  1010.431946, ...,  1009.977417,  1009.834473],
        [ 1010.177063,  1008.479553, ...,  1010.554626,  1010.261353],
        ..., 
        [ 1015.339172,  1015.682373, ...,  1014.277161,  1014.433655],
        [ 1016.094055,  1015.722107, ...,  1014.435425,  1014.611267]],

       ..., 
       [[ 1008.678894,  1009.237427, ...,  1010.142395,  1010.217712],
        [ 1010.003906,  1010.354309, ...,  1010.263489,  1011.105713],
        ..., 
        [ 1011.235962,  1011.026001, ...,  1011.895386,  1011.895081],
        [ 1011.268799,  1011.123596, ...,  1011.9552  ,  1011.97876 ]],

       [[ 1007.1295

### Example using the 'join' method

In [8]:
from netCDF4 import Dataset
from wrf import getvar, ALL_TIMES

file_paths = ["./wrfout_d01_2005-08-28_00:00:00",
              "./wrfout_d01_2005-08-28_12:00:00"]

wrf_files = [Dataset(file_paths[0]),
             Dataset(file_paths[1])]

slp = getvar(wrf_files, "slp", timeidx=ALL_TIMES, method="join")

print (slp)

<xarray.DataArray u'slp' (file: 2, Time: 4, south_north: 73, west_east: 90)>
array([[[[ 1009.792908, ...,  1009.437805],
         ..., 
         [ 1015.951599, ...,  1015.736084]],

        ..., 
        [[ 1010.498413, ...,  1009.797485],
         ..., 
         [ 1015.351562, ...,  1013.459839]]],


       [[[ 1009.027039, ...,  1009.009277],
         ..., 
         [ 1011.268738, ...,  1012.024048]],

        ..., 
        [[ 1007.129578, ...,  1009.516541],
         ..., 
         [ 1010.18573 , ...,  1011.776001]]]], dtype=float32)
Coordinates:
    XLONG     (file, Time, south_north, west_east) float32 -100.775 -100.514 ...
    XLAT      (file, Time, south_north, west_east) float32 17.8033 17.8397 ...
    XTIME     (file, Time) float32 0.0 180.0 360.0 540.0 720.0 900.0 1080.0 ...
  * file      (file) int64 0 1
    datetime  (file, Time) datetime64[ns] 2005-08-28 2005-08-28T03:00:00 ...
Dimensions without coordinates: Time, south_north, west_east
Attributes:
    FieldType:    104
 

## Interpolation Routines

- **interplevel** - linear interpolation to a horizontal plane at a specified height or pressure level
- **vertcross** - vertical cross section interpolation to a vertical plane through two specified points 
  (or a pivot point and angle)
- **vinterp** - interpolates to a "surface", which could be pressure levels or temperature levels like theta-e

### interplevel Example
 
Let's get the 500 hPa heights in decameters
 
``` python
 
from netCDF4 import Dataset
from wrf import getvar, interplevel

file_path = "./wrfout_d01_2005-08-28_00:00:00"

wrf_file = Dataset(file_path)

pres = getvar(wrf_file, "pressure", timeidx=0)
ht = getvar(wrf_file, "z", timeidx=0, units="dm")

ht_500 = interplevel(ht, pres, 500.0)
 
```

In [10]:
import wrf
?wrf.interplevel